# Utilizando web scrapping para descargar la información del DANE

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import openpyxl
from urllib.request import urlopen
import time
import re
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import urllib.request
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from datetime import datetime, timedelta

# Definimos nuestro url principal

url = "https://www.dane.gov.co/index.php/estadisticas-por-tema/agropecuario/sistema-de-informacion-de-precios-sipsa/componente-precios-mayoristas"

In [2]:
#Extraemos todos los href existentes 
webpage = urlopen(url) 

soup = BeautifulSoup(webpage, "html.parser")

links = soup.find_all('a')

"/files/investigaciones/agropecuario/sipsa/anex_\S+.xlsx"
"/index.php/estadisticas-por-tema/agropecuario/sistema-de-informacion-de-precios-sipsa/componente-precios-mayoristas\S+"

In [3]:
# Links de interes en esta primera  prueba, extraidos utilizando expresiones regulares
Anexos = re.findall("/files/investigaciones/agropecuario/sipsa/anex_\S+.xlsx", str(links))
Hiperv = re.findall("/index.php/estadisticas-por-tema/agropecuario/sistema-de-informacion-de-precios-sipsa/componente-precios-mayoristas-\S+", str(soup))


In [ ]:
#for i in range(0, len(Anexos)):
    #Anexos[i] = "https://dane.gov.co/" + Anexos[i]

#for i in range(0, len(Hiperv)):
    #Hiperv[i] = "https://dane.gov.co/" + Hiperv[i]

In [4]:
# Aqui creamos los titles de las referencias en el codigo HTMML
Meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
Años = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

Titulos = []

for i in Meses:
    x = i

    for z in Años:
        Titulos.append("Enlace a Componente Precios Mayoristas SIPSA - " + x + " " + z)

# Los titles no funcionaron para encontrar el botón usando selenium 

#Aqui creamos los xpath bajo el patron encontrado para recorrer las diferentes paginas redireccionadas y descargar los reportes

XPATH_LIST = []
for z in range(1,13):
    for i in range(1,13):

        XPATH_LIST.append(f"//*[@id='cnpv']/pre[{z}]/a[{i}]")

XPATH_LIST_3 = []
for z in range(4,9):
    for i in range(1,13):

        XPATH_LIST_3.append(f"//*[@id='cnpv']/pre[{z}]/span/a[{i}]")

XPATH_LIST_2 = XPATH_LIST[24:]
XPATH_LIST_4 = XPATH_LIST_3[40:]

Anexos_4 = []

for i in range(5,35):
    Anexos_4.append(f"//*[@id='t3-content']/div/article/section/table/tbody/tr[{i}]/td[3]/a")

XPATH_LIST_5 = []
for z in range(8,13):
    for u in range(3,13):
        for i in range(1,13):
            x = f"//*[@id='cnpv']/pre[{z}]/span/span[{u}]/a[{i}]"
            XPATH_LIST_5.append(x)

XPATH_LIST_6 = XPATH_LIST[96:]
XPATH_LIST_7 = XPATH_LIST_6[7:]

#### Utilizamos diferentes listados de xpath debido a diferencias en las especificaciones de los xpath a buscar para redirigir el link o diferencias en las especificaciones de los archivos de interes a descargar en cada pagina

In [ ]:
# Usamos las opciones determinadas de chrome y añadimos opciones especificas para que nos guarde los archivos en una carpeta especifica y descargue automaticamente sin abrir nuevas ventanas
options = webdriver.ChromeOptions()

#Especificamos un folder de descarga donde se guardaran todos los archivos y se modifican las opciones para que descargue sin abrir nuevas ventanas ni preguntar
download_folder = r"C:\Users\Jorge De León\Desktop\Relacion_precios\Archivos_raw"
options.add_experimental_option("prefs", {"download.default_directory": download_folder,
                                                 "download.prompt_for_download": False,
                                                 "download.directory_upgrade": True,
                                                 "safebrowsing.enabled": True})

# Creamos el driver para abrir la pagina
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=options)
browser.get("https://www.dane.gov.co/index.php/estadisticas-por-tema/agropecuario/sistema-de-informacion-de-precios-sipsa/componente-precios-mayoristas")

#Con los links the interes definidos creamos un ciclo para que el programa haga click y descargue cada archivo ya con el HREF identificado para el mes de Mayo desde la pagina principal
#for u in Anexos:
    #download_link = browser.find_element(By.XPATH, f"//a[@href='{u}']")
    #browser.execute_script("arguments[0].click();", download_link)
    #time.sleep(5)

# Ahora recorremos los xpath para ir a las paginas de los diferentes meses a descargar los reportes. 

for i in XPATH_LIST_2:
    try:
        redirect = browser.find_element(By.XPATH, i)
        browser.execute_script("arguments[0].click();", redirect)
        print(f"{i}")

        browser.switch_to.window(browser.window_handles[1])

    except:
        continue

    else:
        current_url = browser.current_url
        webpage_1 = urlopen(current_url) 
        soup_1 = BeautifulSoup(webpage_1, "html.parser")
        links_2 = soup_1.find_all('a')

        Anexos_2 = re.findall("/files/investigaciones/agropecuario/sipsa/mayo\S+.zip", str(links_2))
        Anexos_3 = re.findall("/files/investigaciones/agropecuario/sipsa/\S+.xlsx", str(links_2))

        for z in Anexos_2:
            download_link_2 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_2)
            time.sleep(10)

        for z in Anexos_3:
            download_link_3 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_3)
            time.sleep(10)

        browser.close()
        time.sleep(20)
    
    browser.switch_to.window(browser.window_handles[0])

time.sleep(30)

# Quit the driver
browser.quit()

In [ ]:
# Usamos las opciones determinadas de chrome y añadimos opciones especificas para que nos guarde los archivos en una carpeta especifica y descargue automaticamente sin abrir nuevas ventanas
options = webdriver.ChromeOptions()

#Especificamos un folder de descarga donde se guardaran todos los archivos y se modifican las opciones para que descargue sin abrir nuevas ventanas ni preguntar
download_folder = r"C:\Users\Jorge De León\Desktop\Relacion_precios\Archivos_raw"
options.add_experimental_option("prefs", {"download.default_directory": download_folder,
                                                 "download.prompt_for_download": False,
                                                 "download.directory_upgrade": True,
                                                 "safebrowsing.enabled": True})

# Creamos el driver para abrir la pagina
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=options)
browser.get("https://www.dane.gov.co/index.php/estadisticas-por-tema/agropecuario/sistema-de-informacion-de-precios-sipsa/componente-precios-mayoristas")

#Con los links the interes definidos creamos un ciclo para que el programa haga click y descargue cada archivo ya con el HREF identificado para el mes de Mayo desde la pagina principal
#for u in Anexos:
    #download_link = browser.find_element(By.XPATH, f"//a[@href='{u}']")
    #browser.execute_script("arguments[0].click();", download_link)
    #time.sleep(5)

# Ahora recorremos los xpath para ir a las paginas de los diferentes meses a descargar los reportes. 

for i in XPATH_LIST_4:
    try:
        redirect = browser.find_element(By.XPATH, i)
        redirect_link = redirect.get_attribute("href")
        browser.execute_script(f"window.open('{redirect_link}', '_blank');")
        print(f"{i}")

        browser.switch_to.window(browser.window_handles[1])

        current_url = browser.current_url
        webpage_1 = urlopen(current_url) 
        soup_1 = BeautifulSoup(webpage_1, "html.parser")
        links_2 = soup_1.find_all('a')

        Anexos_2 = re.findall("/files/investigaciones/agropecuario/sipsa/mayo\S+.zip", str(links_2))
        Anexos_3 = re.findall("/files/investigaciones/agropecuario/sipsa/\S+.xlsx", str(links_2))
        Anexos_5 = re.findall("/files/investigaciones/agropecuario/sipsa/\S+.xls", str(links_2))

        for z in Anexos_2:
            download_link_2 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_2)
            time.sleep(10)

        for z in Anexos_3:
            download_link_3 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_3)
            time.sleep(10)

        for z in Anexos_5:
            download_link_5 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_5)
            time.sleep(10)


        browser.close()
        time.sleep(20)

    except:
        continue
    
    browser.switch_to.window(browser.window_handles[0])

time.sleep(30)

# Quit the driver
browser.quit()

In [ ]:
# Usamos las opciones determinadas de chrome y añadimos opciones especificas para que nos guarde los archivos en una carpeta especifica y descargue automaticamente sin abrir nuevas ventanas
options = webdriver.ChromeOptions()

#Especificamos un folder de descarga donde se guardaran todos los archivos y se modifican las opciones para que descargue sin abrir nuevas ventanas ni preguntar
download_folder = r"C:\Users\Jorge De León\Desktop\Relacion_precios\Archivos_raw"
options.add_experimental_option("prefs", {"download.default_directory": download_folder,
                                                 "download.prompt_for_download": False,
                                                 "download.directory_upgrade": True,
                                                 "safebrowsing.enabled": True})

# Creamos el driver para abrir la pagina
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=options)
browser.get("https://www.dane.gov.co/index.php/estadisticas-por-tema/agropecuario/sistema-de-informacion-de-precios-sipsa/componente-precios-mayoristas")

#Con los links the interes definidos creamos un ciclo para que el programa haga click y descargue cada archivo ya con el HREF identificado para el mes de Mayo desde la pagina principal
#for u in Anexos:
    #download_link = browser.find_element(By.XPATH, f"//a[@href='{u}']")
    #browser.execute_script("arguments[0].click();", download_link)
    #time.sleep(5)

# Ahora recorremos los xpath para ir a las paginas de los diferentes meses a descargar los reportes. 

for i in XPATH_LIST_5:
    try:
        redirect = browser.find_element(By.XPATH, i)
        redirect_link = redirect.get_attribute("href")
        browser.execute_script(f"window.open('{redirect_link}', '_blank');")
        print(f"{i}")

        browser.switch_to.window(browser.window_handles[1])

        current_url = browser.current_url
        webpage_1 = urlopen(current_url) 
        soup_1 = BeautifulSoup(webpage_1, "html.parser")
        links_2 = soup_1.find_all('a')

        Anexos_2 = re.findall("/files/investigaciones/agropecuario/sipsa/mayo\S+.zip", str(links_2))
        Anexos_3 = re.findall("/files/investigaciones/agropecuario/sipsa/\S+.xlsx", str(links_2))
        Anexos_5 = re.findall("/files/investigaciones/agropecuario/sipsa/\S+.xls", str(links_2))

        for z in Anexos_2:
            download_link_2 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_2)
            time.sleep(10)

        for z in Anexos_3:
            download_link_3 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_3)
            time.sleep(10)

        for z in Anexos_5:
            download_link_5 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_5)
            time.sleep(10)


        browser.close()
        time.sleep(20)

    except:
        continue
    
    browser.switch_to.window(browser.window_handles[0])

time.sleep(30)

# Quit the driver
browser.quit()

In [ ]:
# Usamos las opciones determinadas de chrome y añadimos opciones especificas para que nos guarde los archivos en una carpeta especifica y descargue automaticamente sin abrir nuevas ventanas
options = webdriver.ChromeOptions()

#Especificamos un folder de descarga donde se guardaran todos los archivos y se modifican las opciones para que descargue sin abrir nuevas ventanas ni preguntar
download_folder = r"C:\Users\Jorge De León\Desktop\Relacion_precios\Archivos_raw"
options.add_experimental_option("prefs", {"download.default_directory": download_folder,
                                                 "download.prompt_for_download": False,
                                                 "download.directory_upgrade": True,
                                                 "safebrowsing.enabled": True})

# Creamos el driver para abrir la pagina
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=options)
browser.get("https://www.dane.gov.co/index.php/estadisticas-por-tema/agropecuario/sistema-de-informacion-de-precios-sipsa/componente-precios-mayoristas")

#Con los links the interes definidos creamos un ciclo para que el programa haga click y descargue cada archivo ya con el HREF identificado para el mes de Mayo desde la pagina principal
#for u in Anexos:
    #download_link = browser.find_element(By.XPATH, f"//a[@href='{u}']")
    #browser.execute_script("arguments[0].click();", download_link)
    #time.sleep(5)

# Ahora recorremos los xpath para ir a las paginas de los diferentes meses a descargar los reportes. 

for i in XPATH_LIST_6:
    try:
        redirect = browser.find_element(By.XPATH, i)
        redirect_link = redirect.get_attribute("href")
        browser.execute_script(f"window.open('{redirect_link}', '_blank');")
        print(f"{i}")

        browser.switch_to.window(browser.window_handles[1])

        current_url = browser.current_url
        webpage_1 = urlopen(current_url) 
        soup_1 = BeautifulSoup(webpage_1, "html.parser")
        links_2 = soup_1.find_all('a')

        Anexos_2 = re.findall("/files/investigaciones/agropecuario/sipsa/mayo\S+.zip", str(links_2))
        Anexos_3 = re.findall("/files/investigaciones/agropecuario/sipsa/\S+.xlsx", str(links_2))
        Anexos_5 = re.findall("/files/investigaciones/agropecuario/sipsa/\S+.xls", str(links_2))

        for z in Anexos_2:
            download_link_2 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_2)
            time.sleep(10)

        for z in Anexos_3:
            download_link_3 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_3)
            time.sleep(10)

        for z in Anexos_5:
            download_link_5 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_5)
            time.sleep(10)


        browser.close()
        time.sleep(20)

    except:
        continue
    
    browser.switch_to.window(browser.window_handles[0])

time.sleep(30)

# Quit the driver
browser.quit()

In [5]:
# Usamos las opciones determinadas de chrome y añadimos opciones especificas para que nos guarde los archivos en una carpeta especifica y descargue automaticamente sin abrir nuevas ventanas
options = webdriver.ChromeOptions()

#Especificamos un folder de descarga donde se guardaran todos los archivos y se modifican las opciones para que descargue sin abrir nuevas ventanas ni preguntar
download_folder = r"C:\Users\Jorge De León\Desktop\Relacion_precios\Archivos_raw"
options.add_experimental_option("prefs", {"download.default_directory": download_folder,
                                                 "download.prompt_for_download": False,
                                                 "download.directory_upgrade": True,
                                                 "safebrowsing.enabled": True})

# Creamos el driver para abrir la pagina
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=options)
browser.get("https://www.dane.gov.co/index.php/estadisticas-por-tema/agropecuario/sistema-de-informacion-de-precios-sipsa/componente-precios-mayoristas")

#Con los links the interes definidos creamos un ciclo para que el programa haga click y descargue cada archivo ya con el HREF identificado para el mes de Mayo desde la pagina principal
#for u in Anexos:
    #download_link = browser.find_element(By.XPATH, f"//a[@href='{u}']")
    #browser.execute_script("arguments[0].click();", download_link)
    #time.sleep(5)

# Ahora recorremos los xpath para ir a las paginas de los diferentes meses a descargar los reportes. 

for i in XPATH_LIST_7:
    try:
        redirect = browser.find_element(By.XPATH, i)
        redirect_link = redirect.get_attribute("href")
        browser.execute_script(f"window.open('{redirect_link}', '_blank');")
        print(f"{i}")

        browser.switch_to.window(browser.window_handles[1])

        current_url = browser.current_url
        webpage_1 = urlopen(current_url) 
        soup_1 = BeautifulSoup(webpage_1, "html.parser")
        links_2 = soup_1.find_all('a')

        Anexos_2 = re.findall("/files/investigaciones/agropecuario/sipsa/mayo\S+.zip", str(links_2))
        Anexos_3 = re.findall("/files/investigaciones/agropecuario/sipsa/\S+.xlsx", str(links_2))
        Anexos_5 = re.findall("/files/investigaciones/agropecuario/sipsa/\S+.xls", str(links_2))

        for z in Anexos_2:
            download_link_2 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_2)
            time.sleep(10)

        for z in Anexos_3:
            download_link_3 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_3)
            time.sleep(10)

        for z in Anexos_5:
            download_link_5 = browser.find_element(By.XPATH, f"//a[@href='{z}']")
            browser.execute_script("arguments[0].click();", download_link_5)
            time.sleep(10)


        browser.close()
        time.sleep(20)

    except:
        continue
    
    browser.switch_to.window(browser.window_handles[0])

time.sleep(30)

# Quit the driver
browser.quit()

C:\Users\Jorge De León\AppData\Local\Temp\ipykernel_14748\1611604774.py:12: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=options)


//*[@id='cnpv']/pre[9]/a[8]
//*[@id='cnpv']/pre[9]/a[9]
//*[@id='cnpv']/pre[9]/a[10]
//*[@id='cnpv']/pre[9]/a[11]
//*[@id='cnpv']/pre[9]/a[12]
